In [2]:
import requests as req
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import psycopg2
import datetime as dt
from tabulate import tabulate
import urllib.request
import ssl
pd.set_option("display.max_columns", None)

### INDICE DE CANASTA BÁSICA A PARTIR DE LOS PRECIOS DE LOS PRODUCTOS EN COTO DIGITAL.

El listado de productos fue obtenido de la pagina del INDEC https://www.indec.gob.ar/ftp/cuadros/sociedad/EPH_metodologia_22_pobreza.pdf y los valores se obtienen de la pagina de Coto Digital https://www.cotodigital3.com.ar/ 
Se utiliza coto digital porque es la cadena de supermercados que tiene mayor cobertura en el país. Podriamos utilizar cualquier otra cadena de supermercados que cuenten con precios publicados en internet.

La idea es tener una referencia de la variacion de los precios de la canasta basica para poder constatarlos contra los datos del INDEC que se publican mensualmente. 
Es una recoleccion muy sencilla de datos sin tener en cuenta ubicacion geoestadistica, por lo que no es una muestra representativa de la poblacion.

Funciones para obtener los productos de la canasta basica de Coto Digital y almacenarlos en un archivo en un diccionario.

In [4]:
listado = {'fecha': dt.datetime.now().strftime("%Y-%m-%d")}

def kilo(nombre_producto, producto_url, porcion = 1):
    valor = BeautifulSoup(producto_url.text, 'html.parser')
    valor = valor.find_all('span', class_='unit')

    
    nombre = nombre_producto.replace(" ", "_")
    
    try:
        valor = valor[0].get_text()
    except IndexError:
        listado.update({nombre: 0})
        print(f'{nombre} IndexError, {listado[nombre]}')
        return None
    
    match = re.search(r"\$([\d,.]+)", valor)
        
    if match:
        number = float(match.group(1).replace(".", "").replace(",", "."))
        print(nombre, (number * porcion)) 
        
        listado.update({nombre: (number * porcion)})    
    else:
        listado.update({nombre: 0})
        print("No se encontró un número en el string")
    
        
    

def unidad(nombre_producto, producto_url):
    valor = BeautifulSoup(producto_url.text, 'html.parser')
    valor = valor.find_all('span', class_='atg_store_newPrice')
    
    nombre = nombre_producto.replace(" ", "_")

    try:
        valor = valor[0].get_text()
    except IndexError:
        listado.update({nombre: 0})
        print(f'{nombre} IndexError, {listado[nombre]}')
        return None
    
    match = re.search(r"\$([\d,.]+)", valor)
        
    if match:
        number = float(match.group(1).replace(".", "").replace(",", "."))
        print(nombre, number)
        listado.update({nombre: number})
            
    else:
        listado.update({nombre: 0})
        print("No se encontró un número en el string")
    





Carga CSV con enlaces a los productos de la canasta basica de Coto Digital

In [5]:
canasta_basica = pd.read_csv('listado canasta basica.csv', sep=';', encoding='latin-1', usecols=['producto', 'cantidad_g_ml', 'url_coto', 'tipo_producto', 'porcion'])
canasta_basica.head(10)

,producto,cantidad_g_ml,url_coto,tipo_producto,porcion
0,Pan Frances,"6,75",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,NaN
1,Galletitas dulces,210,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,NaN
2,Galletitas de agua,420,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,NaN
3,Harina de trigo,"1,08",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,NaN
4,Harina de maiz,210,https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,0.25
5,Arroz,"1,2",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,1.20
6,Fideos secos,"1,74",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,NaN
7,Asado,"1,23",https://www.cotodigital3.com.ar/sitios/cdigi/p...,kilo,1.20
8,Carnaza comun,510,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,0.50
9,Hueso con carne,390,https://www.cotodigital3.com.ar/sitios/cdigi/p...,unidad,0.39


Leemos cada enlace y mediante las funciones de web scraping obtenemos los precios de los productos y lo guardamos en un diccionario.

In [15]:

url = 'https://www.cotodigital3.com.ar'
import urllib.request
import ssl



# Desactivar la verificación SSL (No recomendado para producción)
context = ssl._create_unverified_context()
response = urllib.request.urlopen(url, context=context)
request = req.get(url)
print(response.read())
print(request)

for producto in canasta_basica.index:
  
   if canasta_basica.loc[producto, 'tipo_producto'] == 'kilo':
      url = req.get(canasta_basica.loc[producto, 'url_coto'])
      
      if url.status_code == 200:
         kilo(canasta_basica.loc[producto, 'producto'], url)
      else:
         print('no hay url')
      
   elif canasta_basica.loc[producto, 'tipo_producto'] == 'unidad':
      url = req.get(canasta_basica.loc[producto, 'url_coto'])
      
      if url.status_code == 200:
         unidad(canasta_basica.loc[producto, 'producto'], url)
      else:
         print('no hay url')
      
   else:
      canasta_basica.loc[producto, 'url_coto'] == 'nan'
      print(producto, 'no hay url')

   


b'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n \n \n\n\n\n \n \n \n \n \n \n \n \n \n \n \n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n \n \n\n\n\n\n \n\n\n\n\n\n \n \n \n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n \n \n\n\n\n \n   \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\n\t\n\n\t\n\n\t\n\t\n\n\t\n\t\t\t\n\t\n\t\n\t\n\n\n\n\t\n\t\n\n\t\n\n\t\n\t\n\n\t\n\t\t\t\n\t\n\t\n\t\n\n\n\n\t\n\t\n\n\t\n\n\t\n\t\n\n\t\n\t\t\t\n\t\n\t\n\t\n\n\n\t\n\t\n\t\n\t\n\n\t\n\t\n\t\n\t\n\t\t\t\n\t\n\t\n\t\n\n\n\t\n\t\n\t\n\t\n\n\t\n\t\n\t\n\t\n\t\t\t\n\t\n\t\n\t\n\n \n \n \n \n \n\n\n \n \n\n \n \n \n \n\n\n<!--newLinkBannerHeader -->\n\n \n \n\n \n \n \n \n\n\n\n\t\n\t\n\t\n\t\n\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\n \n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n \n \n\n\n\n\n \n \n \n \n \n \n \n \n \n \n \n \n \n\n\n\n\n\n\n \n \n \n \n\n\n\n\n\n

In [ ]:
listado

{'fecha': '2023-09-20',
 'Pan_Frances': 1830.0,
 'Galletitas_dulces': 269.73,
 'Galletitas_de_agua': 460.98,
 'Harina_de_trigo': 0,
 'Harina_de_maiz': 338.6,
 'Arroz': 0,
 'Fideos_secos': 0,
 'Asado': 2719.9,
 'Carnaza_comun': 1929.9,
 'Hueso_con_carne': 1069.9,
 'Paleta': 2539.9,
 'Carne_picada': 1999.9,
 'Nalga': 3029.9,
 'Higado': 599.9,
 'Pechito_de_cerdo': 1779.9,
 'Pollo': 959.9,
 'Carne_de_pescado': 2990.0,
 'Mortadela': 2092.0,
 'Paleta_cocida': 3759.0,
 'Salchich¢n': 1725.0,
 'Salame': 7093.33,
 'Aceite_de_girasol': 795.28,
 'Margarina_para_cocinar': 413.55,
 'Leche_fluida': 0,
 'Leche_en_polvo_entera': 1775.73,
 'Queso_crema': 272.65,
 'Queso_cuartirolo': 2499.0,
 'Queso_de_rallar': 5139.0,
 'Manteca': 3245.0,
 'Yogur': 0,
 'Dulce_de_leche': 1467.37,
 'Huevo': 2840.0,
 'Manzana': 499.0,
 'Naranja': 499.0,
 'Banana': 739.0,
 'Pera': 249.0,
 'Batata': 1090.0,
 'Papa': 0,
 'Acelga': 199.0,
 'Cebolla': 329.0,
 'Choclo': 1549.0,
 'Lechuga': 699.0,
 'Tomate_perita': 1199.0,
 'Zanah

### Cargar todo a la base de datos postgres de manera local creada en un contenedor docker

Creo el diccionario de productos para la creacion de la tabla. 

In [ ]:
tabla = {'id' : 'SERIAL PRIMARY KEY', 'fecha' : 'DATE' }

for producto in listado:
    if producto == 'fecha':
        continue
    tabla[producto] = 'FLOAT'

tabla

{'id': 'SERIAL PRIMARY KEY', 'fecha': 'DATE'}

Inserto los precios de los productos en la base de datos

In [ ]:
conn = psycopg2.connect(
    host="host.docker.internal",
    database="variacion",
    user="postgres",
    password="postgres" # Cambiar por la contraseña de la base de datos creada
)

In [ ]:
# Conexión a la base de datos
conn = psycopg2.connect(
    host="host.docker.internal",
    database="variacion",
    user="postgres",
    password="postgres" # Cambiar por la contraseña de la base de datos creada
)
# Crear la tabla si no existe
with conn.cursor() as cur:
    cur.execute(
            f"CREATE TABLE IF NOT EXISTS precios ({', '.join([f'{columna} {tipo}' for columna, tipo in tabla.items()])})"
        )
    conn.commit()


# Insertar los datos en la tabla
with conn.cursor() as cur:
    columnas = []
    valores = []
    for columna, valor in listado.items():
        columnas.append(columna)
        valores.append(valor)
        
    data_fecha = cur.execute("SELECT fecha FROM precios WHERE fecha = %s", (valores[0],))
    data_fecha = cur.fetchone()
    print(data_fecha)
    if data_fecha == None:
        print('no hay fecha')
        print('Cargando datos') 
        query = f"INSERT INTO precios ({', '.join(columnas)}) VALUES ({', '.join(['%s'] * len(valores))})"
        cur.execute(query, valores)
        conn.commit()
    else:
        print('Ya se cargaron los datos de hoy')




# Cerrar la conexión a la base de datos
cur.close()
conn.close()

(datetime.date(2023, 3, 27),)
Ya se cargaron los datos de hoy


Traer los productos de la canasta basica desde la tabla de postgres para almacenarlos en CSV o EXCEL. Formatos alternativos para el archivo de salida.

In [ ]:
# Conexión a la base de datos
conn = psycopg2.connect(
    host="host.docker.internal",
    database="variacion",
    user="postgres",
    password="postgres"
)

# Obtener los datos de la tabla
with conn.cursor() as cur:
    cur.execute("SELECT * FROM precios")
    rows = cur.fetchall()
    conn.commit()


if "nan" in rows:
    print("hay un Nan en la tabla")
    
# crear un dataframe si no existe o actualizarlo
try:
    df = pd.read_csv('./datos/precios.csv')
    
    for row in rows:
        print(f"Listado desde guardar_csv_excel \n {row}")
        fecha = row[1].strftime('%Y-%m-%d')
        print(fecha)  
        
        if fecha not in df['fecha'].values:
            df = df.append(pd.DataFrame([row], columns=[desc[0] for desc in cur.description]), ignore_index=True)
            df['fecha'] = pd.to_datetime(df['fecha'])
            df = df.sort_values(by='fecha')
           
            df = df.drop_duplicates()
            df = df.reset_index(drop=True)
            print('actualizando datos')
        else:
            print('no hay datos nuevos')
            continue
    
except FileNotFoundError:
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])
    print('creando dataframe') 
   






#Cerrar la conexión a la base de datos
cur.close()
conn.close()

# Guardar el dataframe en un archivo csv
df.to_csv('./datos/precios.csv', index=False)

# guardar el dataframe en un archivo excel
df.to_excel('./datos/precios.xlsx', index=False)

df.tail(10)

Listado desde guardar_csv_excel 
 (8, datetime.date(2023, 3, 18), 1027.14, 198.49, 349.99, 201.75, 229.02, 223.89, 355.5, 1599.9, 1049.9, 429.9, 1499.9, 999.9, 1299.9, 299.9, 1049.9, 479.9, 1690.0, 1220.0, 1909.0, 1049.0, 4263.33, 574.74, 277.05, 242.15, 1485.55, 264.2, 1469.0, 2699.0, 1592.0, 236.25, 736.12, 1361.0, 499.0, 499.0, 419.0, 199.0, 479.0, 239.0, 329.0, 229.0, 199.0, 899.0, 699.0, 249.0, 199.0, 614.16, 509.04, 1313.15, 304.22, 726.0, 581.26, 316.3, 640.49, 179.6, 0.0, 92.52, 224.67, 117.5, 397.0, 454.0, 830.58, 770.85, 418.56)
2023-03-18
no hay datos nuevos
Listado desde guardar_csv_excel 
 (9, datetime.date(2023, 3, 19), 1027.14, 198.49, 349.99, 201.75, 229.02, 223.89, 355.5, 1599.9, 1049.9, 429.9, 1499.9, 999.9, 1299.9, 299.9, 1049.9, 479.9, 1690.0, 1220.0, 1909.0, 1049.0, 4263.33, 574.74, 277.05, 242.15, 1485.55, 264.2, 1469.0, 2699.0, 1592.0, 236.25, 736.12, 1361.0, 499.0, 499.0, 419.0, 199.0, 479.0, 239.0, 329.0, 229.0, 199.0, 899.0, 699.0, 249.0, 199.0, 614.16, 509.04

,id,fecha,pan_frances,galletitas_dulces,galletitas_de_agua,harina_de_trigo,harina_de_maiz,arroz,fideos_secos,asado,carnaza_comun,hueso_con_carne,paleta,carne_picada,nalga,higado,pechito_de_cerdo,pollo,carne_de_pescado,mortadela,paleta_cocida,salchich¢n,salame,aceite_de_girasol,margarina_para_cocinar,leche_fluida,leche_en_polvo_entera,queso_crema,queso_cuartirolo,queso_de_rallar,manteca,yogur,dulce_de_leche,huevo,manzana,naranja,banana,pera,batata,papa,acelga,cebolla,choclo,lechuga,tomate_perita,zanahoria,zapallo,tomate_envasado,arvejas_en_lata,lentejas_secas,azucar,dulce_de_batata,mermelada,sal_fina,mayonesa,vinagre,caldo_concentrado,gaseosas,jugos_concentrados,soda,cerveza,vino,cafe,yerba,te_en_saquitos
128,144,2023-06-26,1400.28,233.01,398.21,224.33,287.04,0.0,0.0,1799.9,1259.9,699.9,1679.9,1349.9,1999.9,419.9,1249.9,659.9,2299.0,1560.0,2665.0,1325.0,5200.00,687.0,333.85,292.13,1691.17,272.65,1769.0,4229.0,2234.50,0.0,1047.37,2080.0,419.0,379.0,399.0,199.0,849.0,349.0,169.0,299.0,899.0,729.0,999.0,299.0,149.0,0.0,675.0,1989.70,530.46,834.98,786.28,381.68,772.88,218.54,0.0,100.58,295.5,145.13,451.94,0.00,894.9,992.94,450.97
129,145,2023-06-27,1400.28,233.01,398.21,224.33,287.04,0.0,0.0,1799.9,1259.9,699.9,1679.9,1349.9,1999.9,419.9,1249.9,659.9,2399.0,1560.0,2665.0,1325.0,5200.00,687.0,333.85,292.13,1691.17,272.65,1769.0,4229.0,2234.50,0.0,1047.37,2080.0,419.0,379.0,449.0,199.0,849.0,329.0,169.0,249.0,999.0,729.0,999.0,299.0,149.0,0.0,675.0,1989.70,530.46,834.98,786.28,381.68,772.88,218.54,0.0,310.70,295.5,145.13,451.94,517.36,894.9,992.94,450.97
130,146,2023-06-28,1400.28,233.01,398.21,224.33,287.04,0.0,0.0,1899.9,1349.9,749.9,1769.9,1349.9,2099.9,469.9,1249.9,659.9,2399.0,1560.0,2665.0,1325.0,5200.00,687.0,333.85,292.13,1691.17,272.65,1769.0,4229.0,2346.25,0.0,1047.37,2080.0,419.0,379.0,449.0,199.0,849.0,329.0,169.0,249.0,999.0,729.0,899.0,299.0,149.0,0.0,675.0,1989.70,530.46,834.98,786.28,381.68,772.88,218.54,0.0,310.70,295.5,145.13,451.94,517.36,894.9,992.94,450.97
131,147,2023-06-29,1400.28,233.01,398.21,224.33,287.04,0.0,0.0,1899.9,1349.9,749.9,1769.9,1349.9,2099.9,469.9,1349.9,659.9,2399.0,1560.0,2665.0,1325.0,5200.00,687.0,333.85,292.13,1691.17,272.65,1769.0,4229.0,2346.25,0.0,1047.37,2080.0,419.0,379.0,449.0,199.0,849.0,329.0,169.0,199.0,1049.0,749.0,899.0,299.0,149.0,0.0,675.0,1989.70,530.46,834.98,786.28,381.68,772.88,218.54,0.0,310.70,295.5,145.13,451.94,517.36,894.9,992.94,450.97
132,148,2023-06-30,1400.28,233.01,398.21,224.33,287.04,0.0,0.0,1899.9,1349.9,749.9,1769.9,1349.9,2099.9,469.9,1349.9,659.9,1990.0,1560.0,2665.0,1325.0,5200.00,687.0,333.85,292.13,1691.17,272.65,1769.0,4229.0,2346.25,0.0,1047.37,2080.0,419.0,379.0,399.0,199.0,849.0,299.0,169.0,199.0,1049.0,749.0,899.0,299.0,149.0,0.0,675.0,1989.70,530.46,834.98,880.71,381.68,772.88,218.54,0.0,310.70,295.5,145.13,451.94,517.36,894.9,992.94,450.97
133,149,2023-07-01,1400.28,233.01,398.21,224.33,287.04,0.0,0.0,1899.9,1349.9,749.9,1769.9,1349.9,2099.9,469.9,1349.9,659.9,1990.0,1560.0,2665.0,1325.0,5200.00,687.0,333.85,292.13,1691.17,272.65,1769.0,4229.0,2346.25,0.0,1047.37,2080.0,419.0,379.0,399.0,249.0,849.0,299.0,169.0,199.0,1049.0,749.0,899.0,299.0,149.0,0.0,675.0,1989.70,530.46,834.98,880.71,381.68,772.88,218.54,0.0,310.70,295.5,145.13,451.94,517.36,894.9,992.94,450.97
134,150,2023-07-02,1400.28,233.01,398.21,224.33,287.04,0.0,0.0,1899.9,1349.9,749.9,1769.9,1349.9,2099.9,469.9,1349.9,659.9,1990.0,1560.0,2665.0,1325.0,5200.00,687.0,333.85,292.13,1691.17,272.65,1769.0,4229.0,2346.25,0.0,1047.37,2080.0,419.0,379.0,399.0,249.0,849.0,299.0,169.0,199.0,1049.0,749.0,899.0,299.0,149.0,0.0,675.0,1989.70,530.46,834.98,880.71,381.68,772.88,218.54,0.0,100.58,295.5,145.13,451.94,517.36,894.9,992.94,450.97
135,151,2023-07-03,1400.28,244.66,398.21,224.33,287.04,0.0,0.0,1899.9,1349.9,749.9,1769.9,1679.9,2099.9,469.9,1199.9,659.9,1990.0,1950.0,2665.0,1559.0,5200.00,687.0,333.85,292.13,1691.17,272.65,2099.0,3099.0,2346.25,0.0,1047.37,2080.0,419.0,379.0,449.0,249.0,849.0,29

Pequeño analisis para constrastar con los numeros que arrojan los datos en la tabla de Data Studio.

In [ ]:
listado_precios = pd.read_csv('./datos/precios.csv', sep=',', encoding='latin-1')

lista_variaciones = {}
for producto in listado_precios:
    if producto == 'fecha':
        continue
    elif producto == 'id':
        continue
    else:
        variacion = (listado_precios[producto].iloc[-1] / listado_precios[producto].iloc[0] - 1) * 100
        lista_variaciones[producto] = variacion
        
precio_anterior = sum(listado_precios.iloc[0, 2:])
precio_actual = sum(listado_precios.iloc[-1, 2:])
print(f"El precio anterior es ${precio_anterior}")
print(f"El precio actual es ${precio_actual}")
suma = sum(lista_variaciones.values()) / len(lista_variaciones)
print('La variación media de precio de los productos de la canasta básica es del', round(suma, 2), '%')


El precio anterior es $43354.810000000005
El precio actual es $59776.96000000001
La variación media de precio de los productos de la canasta básica es del 40.45 %


### Creamos un csv en formato de lista larga para poder usarlo correctamente en data studio.

El formato de lista larga nos permite tener un registro por producto y por fecha y facilitar su visibilidad en data studio.

In [ ]:
lista_tabla = df.columns.to_list()

lista_tabla.remove('id')
lista_tabla.remove('fecha')
lista_tabla




NameError: name 'df' is not defined

In [ ]:
lista_larga = df.melt(id_vars=['fecha'], value_vars=lista_tabla, var_name='producto', value_name='precio')
lista_larga[-2:]['producto']

8692    te_en_saquitos
8693    te_en_saquitos
Name: producto, dtype: object

In [ ]:
lista_larga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8694 entries, 0 to 8693
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fecha     8694 non-null   object 
 1   producto  8694 non-null   object 
 2   precio    8694 non-null   float64
dtypes: float64(1), object(2)
memory usage: 203.9+ KB


In [ ]:
lista_larga.to_csv('./datos/precios_lista_larga.csv', index=False)
lista_larga['fecha'] = pd.to_datetime(lista_larga['fecha'])

### Subimos la lista_larga a una DDBB en postgres online. 
 

la base esta en azure

In [ ]:
fecha = dt.datetime.now().strftime("%Y-%m-%d")
fecha

'2023-07-05'

In [ ]:
from certificados_ddbb import ddbb_pass, user, host
print(ddbb_pass, user, host)

coto_bot_laimas postgres canasta-2.crdqtsbdpist.us-east-2.rds.amazonaws.com


In [ ]:
print(type(lista_larga.loc[1, "fecha"].strftime("%Y-%m-%d")))

<class 'str'>


In [ ]:


# Conexión a la base de datos
conn = psycopg2.connect(
    host=host,
    database="postgres",
    user=user,
    password=ddbb_pass
)

# Crear la tabla si no existe
with conn.cursor() as cur:
    
    cur.execute(f"CREATE TABLE IF NOT EXISTS precios_lista_larga (fecha DATE, producto VARCHAR(255), precio FLOAT, id SERIAL PRIMARY KEY)")
    conn.commit()

# Insertar los datos en la tabla



    for i in lista_larga.index:
            


            if lista_larga.loc[i, 'fecha'].strftime("%Y-%m-%d") == dt.datetime.now().strftime("%Y-%m-%d"):
                
                print('es hoy')
                producto_fecha = cur.execute("SELECT fecha, producto FROM precios_lista_larga WHERE fecha = %s AND producto = %s", (lista_larga.loc[i, 'fecha'], lista_larga.loc[i, 'producto']))
                producto_fecha = cur.fetchone()
                print(producto_fecha)
            
                if producto_fecha == None:
                    print(f'no existe, cargando {lista_larga.loc[i, "producto"]}')
                        
                    cur.execute("INSERT INTO precios_lista_larga (fecha, producto, precio) VALUES (%s, %s, %s)", (lista_larga.loc[i, 'fecha'], lista_larga.loc[i, 'producto'], lista_larga.loc[i, 'precio']))
                    conn.commit()
                else:
                    print(f'ya existe {lista_larga.loc[i, "producto"]}, {lista_larga.loc[i, "fecha"]}')
                    continue
            else:
                
                continue



#Cerrar la conexión a la base de datos
cur.close()
conn.close()

print('Datos cargados')
    

es hoy
(datetime.date(2023, 7, 5), 'pan_frances')
ya existe pan_frances, 2023-07-05 00:00:00
es hoy
(datetime.date(2023, 7, 5), 'galletitas_dulces')
ya existe galletitas_dulces, 2023-07-05 00:00:00
es hoy
(datetime.date(2023, 7, 5), 'galletitas_de_agua')
ya existe galletitas_de_agua, 2023-07-05 00:00:00
es hoy
(datetime.date(2023, 7, 5), 'harina_de_trigo')
ya existe harina_de_trigo, 2023-07-05 00:00:00
es hoy
(datetime.date(2023, 7, 5), 'harina_de_maiz')
ya existe harina_de_maiz, 2023-07-05 00:00:00
es hoy
(datetime.date(2023, 7, 5), 'arroz')
ya existe arroz, 2023-07-05 00:00:00
es hoy
(datetime.date(2023, 7, 5), 'fideos_secos')
ya existe fideos_secos, 2023-07-05 00:00:00
es hoy
(datetime.date(2023, 7, 5), 'asado')
ya existe asado, 2023-07-05 00:00:00
es hoy
(datetime.date(2023, 7, 5), 'carnaza_comun')
ya existe carnaza_comun, 2023-07-05 00:00:00
es hoy
(datetime.date(2023, 7, 5), 'hueso_con_carne')
ya existe hueso_con_carne, 2023-07-05 00:00:00
es hoy
(datetime.date(2023, 7, 5), 'pale

In [ ]:
conn = psycopg2.connect(
    host=host,
    database="postgres",
    user=user,
    password=ddbb_pass
)


In [ ]:

# Insertar los datos en la tabla
with conn.cursor() as cur:
    for i in lista_larga.index:
        if lista_larga.loc[i, 'fecha'].strftime("%Y-%m-%d"):
            producto_fecha = cur.execute("SELECT fecha, producto FROM precios_lista_larga WHERE fecha = %s AND producto = %s", (lista_larga.loc[i, 'fecha'], lista_larga.loc[i, 'producto']))
            producto_fecha = cur.fetchone()
            print(producto_fecha)
                
            if producto_fecha == None:
                print(f'no existe, cargando {lista_larga.loc[i, "producto"]}')
                            
                cur.execute("INSERT INTO precios_lista_larga (fecha, producto, precio) VALUES (%s, %s, %s)", (lista_larga.loc[i, 'fecha'], lista_larga.loc[i, 'producto'], lista_larga.loc[i, 'precio']))
                conn.commit()
            else:
                print(f'ya existe {lista_larga.loc[i, "producto"]}, {lista_larga.loc[i, "fecha"]}')
                continue
        else:        
            continue 


#Cerrar la conexión a la base de datos
cur.close()
conn.close()

print('Datos cargados')
                




None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
None
no existe, cargando pan_frances
N

In [ ]:
from datetime import datetime, timedelta, timezone

hora_actual = datetime.now(timezone.utc).hour
print(hora_actual)

def horario_branch_check():
    if hora_actual < 15 + 3 and hora_actual >= 8 + 3:
        print('es horario')
        
    else:
        print('no es horario')
        
horario_branch_check()

11
es horario


In [ ]:
#Creo una lista larga con fecha, producto, precio a partir de la lista listado

lista_larga = pd.DataFrame(columns=['fecha', 'producto', 'precio'])

for producto in listado:
    if producto == 'fecha':
        continue
    else:
        lista_larga = pd.concat([lista_larga, pd.DataFrame({'fecha': [listado['fecha']], 'producto': [producto], 'precio': [listado[producto]]})], ignore_index=True)

#nombre producto a minuscula
lista_larga['producto'] = lista_larga['producto'].str.lower()

lista_larga["fecha"] = pd.to_datetime(lista_larga["fecha"])

lista_larga.head(10)


,fecha,producto,precio
0,2023-09-21,pan_frances,1830.00
1,2023-09-21,galletitas_dulces,269.73
2,2023-09-21,galletitas_de_agua,460.98
3,2023-09-21,harina_de_trigo,0.00
4,2023-09-21,harina_de_maiz,338.60
5,2023-09-21,arroz,0.00
6,2023-09-21,fideos_secos,0.00
7,2023-09-21,asado,2719.90
8,2023-09-21,carnaza_comun,1929.90
9,2023-09-21,hueso_con_carne,1069.90


In [36]:
datos = lista_larga

with conn.cursor() as cur:
        cur.execute(
            f"CREATE TABLE IF NOT EXISTS precios_lista_larga (fecha DATE, producto VARCHAR(255), precio FLOAT, id SERIAL PRIMARY KEY)"
        )
        conn.commit()

        # Insertar los datos en la tabla

        for i in datos.index:
            if datos.loc[i, "fecha"].strftime("%Y-%m-%d") == dt.datetime.now().strftime(
                "%Y-%m-%d"
            ):
                producto_fecha = cur.execute(
                    "SELECT fecha, producto FROM precios_lista_larga WHERE fecha = %s AND producto = %s",
                    (datos.loc[i, "fecha"], datos.loc[i, "producto"]),
                )
                producto_fecha = cur.fetchone()
                print(producto_fecha)

                if producto_fecha == None:
                    print(f'no existe, cargando {datos.loc[i, "producto"]}')

                    cur.execute(
                        "INSERT INTO precios_lista_larga (fecha, producto, precio) VALUES (%s, %s, %s)",
                        (
                            datos.loc[i, "fecha"],
                            datos.loc[i, "producto"],
                            datos.loc[i, "precio"],
                        ),
                    )
                    conn.commit()
                else:
                    print(
                        f'ya existe {datos.loc[i, "producto"]}, {datos.loc[i, "fecha"]}'
                    )
                    continue
            else:
                continue

    # Cerrar la conexión a la base de datos
cur.close()
conn.close()

print("DATOS")

NameError: name 'conn' is not defined

In [48]:
# fechas variables
fecha_actual = dt.datetime.now()
fecha = dt.datetime.now().strftime("%Y-%m-%d")
print(fecha)

primer_dia_mes_actual = dt.datetime.now().replace(day=1).strftime("%Y-%m-%d")


# fin de mes
primer_dia_siguiente_mes = dt.date(fecha_actual.year, fecha_actual.month + 1, 1)
print(primer_dia_siguiente_mes)
ultimo_dia_mes_actual = primer_dia_siguiente_mes - dt.timedelta(days=1)
print(ultimo_dia_mes_actual)
# Verificar si el día actual es el último día del mes
es_fin_de_mes = fecha == ultimo_dia_mes_actual.strftime("%Y-%m-%d")

print(es_fin_de_mes)

# primero de mes
primer_dia = fecha == primer_dia_mes_actual
print(primer_dia)


# nombre del mes en español


meses = {
    "January": "Enero",
    "February": "Febrero",
    "March": "Marzo",
    "April": "Abril",
    "May": "Mayo",
    "June": "Junio",
    "July": "Julio",
    "August": "Agosto",
    "September": "Septiembre",
    "October": "Octubre",
    "November": "Noviembre",
    "December": "Diciembre",
}

nombre_mes = meses[dt.datetime.now().strftime("%B")]
print(nombre_mes)



# Obtener la semana del año
semana_del_año = fecha_actual.isocalendar()[1]
print(f"Semana del año: {semana_del_año}")

# Obtener la primer fecha de la semana actual
primer_dia_semana_actual = fecha_actual - dt.timedelta(days=fecha_actual.weekday())

print(f"Primer día de la semana actual: {primer_dia_semana_actual}")

# Obtener la primer fecha de la semana pasada
primer_dia_semana_pasada = primer_dia_semana_actual - dt.timedelta(weeks=1)
primer_dia_semana_pasada = primer_dia_semana_pasada.strftime("%Y-%m-%d")
print(f"Primer día de la semana pasada: {primer_dia_semana_pasada}")

# Obtener la última fecha de la semana pasada (6 días antes del primer día de la semana actual)
ultimo_dia_semana_pasada = primer_dia_semana_actual - dt.timedelta(days=1)
ultimo_dia_semana_pasada = ultimo_dia_semana_pasada.strftime("%Y-%m-%d")
print(f"Último día de la semana pasada: {ultimo_dia_semana_pasada}")


# cargamos el csv de precios
try:
    lista_larga = pd.read_csv("/opt/airflow/data/precios_lista_larga.csv")
except:
    lista_larga = pd.read_csv(
        r"C:\codeo_juego\scrapping_coto_canasta_basica\airflow\data\precios_lista_larga.csv"
    )

2023-09-25
2023-10-01
2023-09-30
False
False
Septiembre
Semana del año: 39
Primer día de la semana actual: 2023-09-25 12:52:43.323816
Primer día de la semana pasada: 2023-09-18
Último día de la semana pasada: 2023-09-24


In [42]:

# precios del primer dia del mes actual y del dia actual
def precios(lista, dia1, dia2):
    # Precios del primer día del mes actual
    if fecha == lista["fecha"].max():
        precios_dia1 = lista_larga[
            lista_larga["fecha"] == dia1
        ]

        # precios del dia actual
        precios_dia2 = lista_larga[lista_larga["fecha"] == dia2]

        
        return precios_dia1, precios_dia2

    else:
        print("No hay precios del dia actual")
        return False, False


# limpio los precios
def limpio_precios(dia1, dia2):
    if dia1 is False or dia2 is False:
        print("No hay precios del dia actual")
        return False, False
    else:
        mask1 = dia2["precio"] == 0
        mask2 = dia2.groupby("producto")["precio"].transform("first") == 0
        mask3 = dia1["precio"] == 0
        mask4 = dia1.groupby("producto")["precio"].transform("first") == 0

        # Eliminar los productos sin precio
        dia1 = dia1[dia1["precio"] != 0]
        dia1 = dia1[
            dia1["producto"].isin(dia2.loc[~mask2, "producto"])
        ]

        dia2 = dia2[dia2["precio"] != 0]
        dia2 = dia2[
            dia2["producto"].isin(dia1.loc[~mask4, "producto"])
        ]

        # Restablecer los índices
        dia1.reset_index(drop=True, inplace=True)
        dia2.reset_index(drop=True, inplace=True)

        return dia1, dia2


# saco la variacion de precios mensual al dia de la fecha
def variacion_precios(precios_limpios):
    if precios_limpios[0] is False or precios_limpios[1] is False:
        print("No hay precios del dia actual")
        return False
    else:
        precio_anterior = sum(precios_limpios[0]["precio"])
        precio_actual = sum(precios_limpios[1]["precio"])

        # calculo la variación

        variacion = round((precio_actual / precio_anterior - 1) * 100, 2)
        promedio_precio = round(
            (precio_actual + precio_anterior) / 2, 2
        )
        return variacion, promedio_precio


def variacion(dia1, dia2):
    variacion_personalizada = variacion_precios(limpio_precios(precios(lista_larga, dia1, dia2)[0], precios(lista_larga, dia1, dia2)[1]))
    return variacion_personalizada[0], variacion_personalizada[1]
    

In [49]:
ultima_semana = variacion(primer_dia_semana_pasada, ultimo_dia_semana_pasada)
print(ultima_semana)
primer_dia_semana_pasada = primer_dia_semana_actual - dt.timedelta(weeks=1)
ultimo_dia_semana_pasada = primer_dia_semana_actual - dt.timedelta(days=1)
semana_anterior=variacion(
                        (primer_dia_semana_pasada - dt.timedelta(weeks=1)).strftime("%Y-%m-%d"),
                        (ultimo_dia_semana_pasada - dt.timedelta(weeks=1)).strftime("%Y-%m-%d"))
                    
print(semana_anterior)

diferencia_entre_semanas = round((ultima_semana[1] / semana_anterior[1] - 1) * 100, 2)
print(diferencia_entre_semanas)

(-1.49, 73902.49)
(0.17, 74471.87)
-0.76


In [11]:


if primer_dia_semana_actual == dt.datetime.now().strftime("%Y-%m-%d"):
    variacion_semanal=variacion(primer_dia_semana_pasada, ultimo_dia_semana_pasada)
    print(f'Primer dia de la semana {semana_del_año}')
else:
    print(f'No es el primer dia de la semana actual {semana_del_año}')
    print(f'pasaron {primer_dia_semana_actual - dt.datetime.now()} días')


No es el primer dia de la semana actual 39
pasaron -1 day, 23:56:18.862439 días


In [10]:
url = 'https://www.cotodigital3.com.ar'
import urllib.request
import ssl



# Desactivar la verificación SSL (No recomendado para producción)
context = ssl._create_unverified_context()
response = urllib.request.urlopen(url, context=context)

print(response.read())

b'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n \n \n\n\n\n \n \n \n \n \n \n \n \n \n \n \n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n \n \n\n\n\n\n \n\n\n\n\n\n \n \n \n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n \n \n\n\n\n \n   \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\n\t\n\n\t\n\n\t\n\t\n\n\t\n\t\t\t\n\t\n\t\n\t\n\n\n\n\t\n\t\n\n\t\n\n\t\n\t\n\n\t\n\t\t\t\n\t\n\t\n\t\n\n\n\n\t\n\t\n\n\t\n\n\t\n\t\n\n\t\n\t\t\t\n\t\n\t\n\t\n\n\n\t\n\t\n\t\n\t\n\n\t\n\t\n\t\n\t\n\t\t\t\n\t\n\t\n\t\n\n\n\t\n\t\n\t\n\t\n\n\t\n\t\n\t\n\t\n\t\t\t\n\t\n\t\n\t\n\n \n \n \n \n \n\n\n \n \n\n \n \n \n \n\n\n<!--newLinkBannerHeader -->\n\n \n \n\n \n \n \n \n\n\n\n\t\n\t\n\t\n\t\n\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\n \n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n \n \n\n\n\n\n \n \n \n \n \n \n \n \n \n \n \n \n \n\n\n\n\n\n\n \n \n \n \n\n\n\n\n\n